In [1]:
# This requires you to have pins 567 up to flash
%esptool erase

Executing:
  esptool.py --port /dev/ttyUSB0 erase_flash

esptool.py v2.8
Serial port /dev/ttyUSB0
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266EX
Features: WiFi
Crystal is 26MHz
MAC: 68:c6:3a:99:41:c2
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 8.3s
Hard resetting via RTS pin...


In [2]:
# This requires you to have pins 567 up to flash
# This firmware was built with mqtt_as from https://github.com/goatchurchprime/jupyter_micropython_developer_notebooks/blob/master/mqtt_async_projects/esp8266_commissioning_mqtt_as.ipynb
%esptool esp8266 bin/firmware-combined.bin

Executing:
  esptool.py --port /dev/ttyUSB0 --baud 460800 write_flash --flash_size=detect -fm dio 0 bin/firmware-combined.bin

esptool.py v2.8
Serial port /dev/ttyUSB0
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266EX
Features: WiFi
Crystal is 26MHz
MAC: 68:c6:3a:99:41:c2
Uploading stub...
Running stub...
Stub running...
Changing baud rate to 460800
Changed.
Configuring flash size...
Auto-detected Flash size: 4MB
Flash params set to 0x0240
Compressed 624400 bytes to 406630...
Wrote 624400 bytes (406630 compressed) at 0x00000000 in 9.3 seconds (effective 535.4 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


In [3]:
# now just the 56 switches
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [4]:
help("modules")

__main__          lighthttpserve_funcs                uasyncio/queues   urandom
_boot             lwip              uasyncio/synchro  ure
_onewire          machine           ubinascii         uselect
_webrepl          math              ucollections      usocket
apa102            micropython       ucryptolib        ussl
btree             mqtt_as           uctypes           ustruct
builtins          mqtt_as_timeout   uerrno            utime
collections/deque neopixel          uhashlib          utimeq
dht               network           uheapq            uwebsocket
ds18x20           ntptime           uio               uzlib
esp               onewire           ujson             webrepl
flashbdev         port_diag         umqtt/robust      webrepl_setup
framebuf          sys               umqtt/simple      websocket_helper
gc                uarray            uos
inisetup          uasyncio/__init__ upip
itertools         uasyncio/core     upip_utarfile
Plus any modules on the filesystem


In [9]:
# this is the code for changing the wifi credentials
%sendtofile --source utils.py
%sendtofile --source confighotspot.py

Sent 93 lines (2923 bytes) to confighotspot.py.


In [8]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   sensorcity.io
pinled       14
boardname    abdushop8
Dserialsource uart0
serialsource none

Sent 7 lines (154 bytes) to config.txt.


In [7]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [97]:
%sendtofile main.py

import time, network, os, sys, itertools
from machine import Pin, UART
from mqtt_as import config, MQTTClient
import uasyncio as asyncio

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
    
shortmacaddress ="{:02x}{:02x}{:02x}".format(*network.WLAN().config("mac")[-3:])
print("*** connected shortmac", shortmacaddress)
boardname = fconfig["boardname"]
topicstatus = "ab/%s/%s/status"%(boardname, shortmacaddress)
topicserial = "ab/%s/%s/serial"%(boardname, shortmacaddress)

async def serialtask(uart):
    buf = memoryview(bytearray(240))
    e = 0
    while True:
        s, e = e, e + (uart and uart.readinto(buf[e:]) or 0)
        while s < e:
            s += 1
            if buf[s-1] == ord('\n'):
                if client.isconnected():
                    await client.publish(topicserial, buf[:s-1])
                    print("message sent")
                buf[:e-s] = buf[s:e]
                s, e = 0, e-s
        if e == len(buf):
            if client.isconnected():
                await client.publish(topicstatus, b"BUFFEROVERFLOW")
            e = 0
        await asyncio.sleep_ms(5)

        
async def connecttaskrobust():
    for i in range(5):
        print("*** connecttaskrobust", i)
        try:
            await client.connect()
            print("got broker connection to", client._addr)
            return True
        except OSError as e:
            print("client connect error", [e])
            if not client._sta_if.isconnected():
                print("wifi failed to connect to", (fconfig["wifiname"], fconfig["wifipassword"]))
                if i == 1:
                    return False
            elif client._addr is None:
                print("dns failed for", fconfig["mqttbroker"])
            else:
                import uerrno
                errorcode = uerrno.errorcode.get(e.args[0], e.args[0])
                print("connection failure errorcode", errorcode)
        await asyncio.sleep_ms(500)
    return False

def handleconnection_indirect(reader, writer):
    try:
        return handleconnection(reader, writer)
    except OSError as e:
        print("handleconnection exception", str(e))

async def connecttask():
    print("*** connecttask")
    if not (await connecttaskrobust()):
        from confighotspot import handleconnection
        si = network.WLAN(network.AP_IF)
        si.active(True)
        ipnumber = si.ifconfig()[0]
        print("Creating access point MicroPython-"+shortmacaddress)
        print("* Running on http://%s/" % ipnumber)
        aloop.create_task(asyncio.start_server(handleconnection_indirect, ipnumber, 80))
        return
    
    print("*** connected")
    await client.publish(topicstatus, "connected", retain=True)
    for i in range(19):
        pinled.value(i%2)
        await asyncio.sleep_ms(100)

    uart = None
    if fconfig["serialsource"] == "uart0":
        uart = UART(0, baudrate=115200, rxbuf=240)
        os.dupterm(None, 1)

    aloop.create_task(serialtask(uart))
    for i in itertools.count():
        print("connecttask loop", i, client.isconnected())
        if client.isconnected():
            await client.publish(topicstatus, "connected %d"%i, retain=True)
        else:
            print("disconnected", i)
        await asyncio.sleep_ms(3000)

async def startupdisconnectedheartbeattask():
    for i in itertools.count():
        print("startupheartbeattask", i)
        pinled.value(i%2)
        await asyncio.sleep_ms(1000)
        if client.isconnected():
            break
    print("startupheartbeattask connected", client._addr)
            
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['will'] = [ topicstatus, "dropped", True]
  
client = MQTTClient(config)
client._addr = None

aloop = asyncio.get_event_loop()
aloop.create_task(startupdisconnectedheartbeattask())
aloop.create_task(connecttask())
aloop.run_forever()



Sent 116 lines (3993 bytes) to main.py.


In [ ]:
      digitalWrite(datapin, 0); 
      delay(50); 
      for (int i = 0; i < 100; i++) {
        digitalWrite(datapin, 1); 
        delay(switchvalues[i] ? 5 : 2); 
        digitalWrite(datapin, 0); 
        delay(4); 
      }
      digitalWrite(datapin, 1); 
machine.time_pulse_us

In [108]:
%serialconnect

Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [109]:
from machine import Pin, time_pulse_us
import time
p = Pin(13, Pin.IN)


In [110]:

time_pulse_us(p, 1, 1000000)
for j in range(190):
    for i in range(30):
        time.sleep_us(100)
        print(p.value(), end="")


0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [129]:
x = time_pulse_us(p, 0, 500000)
print(x)

50015


In [132]:
print(bytearray(10))


bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')


In [137]:
dat = bytearray(100)
dat[0] = ord('1')
print(dat)

bytearray(b'1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')


In [146]:
dat = bytearray(100)
def readpulses():
    x = (time_pulse_us(p, 0, 500000)+500)//1000
    if x != 50:
        return ""
    for i in range(100):
        x = (time_pulse_us(p, 1, 10000)+500)//1000
        if x == 2:
            dat[i] = ord('0')
        elif x == 5:
            dat[i] = ord('1')
        elif i != 0:
            print(dat[:i], x)
            return ""
    return dat


In [147]:
while 1:
    x = readpulses()
    if x:`
        print(x)

# minimal connection below

bytearray(b'1111111111111111111110111111111111111111111111111111111111111111111111111111111111111111111111111111')
bytearray(b'1111111111111111111110111111111111111111111111111111111111111111111111111111111111111111111111111111')
bytearray(b'1111111111111111111110111111111111111111111111111111111111111111111111111111111111111111111111111111')
.bytearray(b'1111111111111111111110111111111111111111111111111111111111111111111111111111111111111111111111111111')
bytearray(b'1111111111111111111110111111111111111111111111111111111111111111111111111111111111111111111111111111')
bytearray(b'1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111')
bytearray(b'1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111')
bytearray(b'1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111')
.bytearray(b'111111111111111111111111111111111111111111111111111111111111111111

Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
  File "<stdin>", line 5, in readpulses
KeyboardInterrupt: 


In [93]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [94]:
%sendtofile main.py

import network, socket, time, ubinascii, itertools

w = network.WLAN()
w.active(1)
shortmacaddress = ubinascii.hexlify(w.config('mac')[-3:]).decode()
#w.connect("DoESLiverpool", "decafbad00")
w.connect("AndroidAP01BD", "mattisatwat")
print("*** connected shortmac", shortmacaddress)
topicstatus = "ab/%s/%s/status"%("unknown", shortmacaddress)

try:
    print(socket.getaddrinfo("sensorcity.io", 80))
except OSError as e:
    import uerrno
    errorcode = uerrno.errorcode.get(e.args[0], e.args[0])
    print(errorcode)
    
import machine
from umqtt.robust import MQTTClient
client = MQTTClient(machine.unique_id(), "sensorcity.io", keepalive=60)
client.connect()
for i in itertools.count():
    print("clientstatus", i)
    client.publish(topicstatus, str(i))
    time.sleep_ms(1000)



Sent 26 lines (787 bytes) to main.py.


In [60]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [61]:
%sendtofile --source confighotspot.py

Sent 93 lines (2923 bytes) to confighotspot.py.

[missing-OK]ip:10.0.36.143,mask:255.0.0.0,gw:10.0.0.1


[Late OK]



In [62]:
%sendtofile dmain.py

import time, os, network
import uasyncio as asyncio
from confighotspot import handleconnection

si = network.WLAN(network.AP_IF) # create access-point interface
shortmacaddress ="{:02x}{:02x}{:02x}".format(*si.config("mac")[-3:])
si.active(True)         # activate the interface
print("Creating access point MicroPython-"+shortmacaddress)

ipnumber = si.ifconfig()[0]
print("Device has ipnumber", ipnumber)

def handleconnection_indirect(reader, writer):
    try:
        return handleconnection(reader, writer)
    except OSError as e:
        print("handleconnection exception", str(e))


port = 80
loop = asyncio.get_event_loop()
print("* Running on http://%s:%s/" % (ipnumber, port))
loop.create_task(asyncio.start_server(handleconnection_indirect, ipnumber, port))
loop.run_forever()


Sent 24 lines (789 bytes) to dmain.py.


In [63]:
import dmain

Creating access point MicroPython-0f263f
Device has ipnumber 192.168.4.1
* Running on http://192.168.4.1:80/
.add 1
aid 1
station: 00:28:f8:a0:5f:4b join, AID = 1
.............
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

